# Testing Hardcover API with GraphQL

In [13]:
import requests
import pandas as pd
import os

In [11]:
query = """
query {
  me {
    username
    books_count
    lists(limit: 50) {
      books_count
      list_books {
        user_books {
          user_book_reads {
            finished_at
          }
        }
      }
    }
  }
}
"""
url = "https://api.hardcover.app/v1/graphql"

headers = {
    "authorization": f"Bearer {os.environ['TOKEN']}",
    "content-type": "application/json"
}


In [ ]:
response = requests.post(url, json={'query': query}, headers=headers)

data = response.json()
data

# Hardcover get user lists

In [39]:
# Get my user id:
id_query = """
query id {
    me {
        id
    }
}
"""

response = requests.post(url, json={'query': id_query}, headers=headers)

In [41]:
response.json()

{'data': {'me': [{'id': 55022}]}}

In [ ]:
query = """
query GetUserLists {
  lists(
      where: {user_id: {_eq: 55022}},
      order_by: {updated_at: desc}
  ) {
      id
      name
      description
      books_count
      public
      created_at
      updated_at
  }
}
"""

query = """
query UserBooks {
    user_books(
          where: {
              user_id: {_eq: 55022}
          },
          distinct_on: book_id
          limit: 5
          offset: 0
    ) {
      book {
            title
            pages
            release_date
      }
    }
}
"""

In [64]:
response = requests.post(url, json={'query': query}, headers=headers)
response.status_code

200

In [65]:
response.json()

{'errors': [{'message': 'not a valid graphql query',
   'extensions': {'path': '$.query', 'code': 'validation-failed'}}]}

## Notes on Hardcover API: <p>
- Only public lists (shelves) can be accessed. <br>
- Workflow for getting a user's shelves is:
  - Get all their lists 
  - Get books in each list by running a separate query for each list (or maybe all lists can be included in one query, but have to be specified separately)

In [ ]:
import streamlit as st
import urllib
import xmltodict

In [ ]:
user_input = "https://www.goodreads.com/user/show/33873248-khadeeja-ahmed"
user_id = "".join(filter(lambda i: i.isdigit(), user_input))
user_name = user_input.split(user_id, 1)[1].split("-", 1)[1].replace("-", " ")


'khadeeja ahmed'

In [ ]:
@st.cache_data
def get_user_data(user_id, key=gr_key, v="2", shelf="read", per_page="200"):
    api_url_base = "https://www.goodreads.com/review/list/"
    final_url = (
        api_url_base
        + user_id
        + ".xml?key="
        + key
        + "&v="
        + v
        + "&shelf="
        + shelf
        + "&per_page="
        + per_page
    )
    contents = urllib.request.urlopen(final_url).read()
    return contents


user_input = str(user_input)    # Taking input earlier in Streamlit
contents = get_user_data(user_id=user_id, v="2", shelf="read", per_page="200")  # User id taken earlier
contents = xmltodict.parse(contents)